# Variaciones Information Value

En esta notebook vemos las distintas variantes de Information Value y cuánto laburo extra demandarían

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("../data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [2]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("../output/provinces_words.csv", index_col=0)

cant_palabras = [c for c in df.columns if re.match(r'.*ocurrencias$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*usuarios$', c)]

print("columnas de palabras => {}\n".format(cant_palabras))
print("columnas de usuarios => {}".format(cant_personas))

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)
df["cant_palabra"] = df[cant_palabras].sum(axis=1)
df["cant_usuarios"] = df[cant_personas].sum(axis=1)

columnas de palabras => ['buenosaires_ocurrencias', 'catamarca_ocurrencias', 'chaco_ocurrencias', 'chubut_ocurrencias', 'cordoba_ocurrencias', 'corrientes_ocurrencias', 'entrerios_ocurrencias', 'formosa_ocurrencias', 'jujuy_ocurrencias', 'lapampa_ocurrencias', 'larioja_ocurrencias', 'mendoza_ocurrencias', 'misiones_ocurrencias', 'neuquen_ocurrencias', 'rionegro_ocurrencias', 'salta_ocurrencias', 'sanjuan_ocurrencias', 'sanluis_ocurrencias', 'santacruz_ocurrencias', 'santafe_ocurrencias', 'santiago_ocurrencias', 'tierradelfuego_ocurrencias', 'tucuman_ocurrencias']

columnas de usuarios => ['buenosaires_usuarios', 'catamarca_usuarios', 'chaco_usuarios', 'chubut_usuarios', 'cordoba_usuarios', 'corrientes_usuarios', 'entrerios_usuarios', 'formosa_usuarios', 'jujuy_usuarios', 'lapampa_usuarios', 'larioja_usuarios', 'mendoza_usuarios', 'misiones_usuarios', 'neuquen_usuarios', 'rionegro_usuarios', 'salta_usuarios', 'sanjuan_usuarios', 'sanluis_usuarios', 'santacruz_usuarios', 'santafe_usuario

In [4]:
df = df[(df.cant_palabra > 75) & (df.cant_usuarios >= 8) ]

Calculemos el valor de la información de cada palabra.

In [5]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log2(23) - df["entropy_palabras"]
df["delta_personas"] = np.log2(23) - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]
df["ival_norm"] = (1+df["ival_palabras"]) * (1+df["ival_personas"])

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)


Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [6]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Palabras candidatas = {}".format(candidatas))
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [7]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp,
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 200
Total analizadas actualmente = 795
Faltan etiquetar 205 palabras
Porcentaje candidatas = 0.25157232704402516
########################################
Métrica: Personas
Palabras candidatas = 265
Total analizadas actualmente = 796
Faltan etiquetar 204 palabras
Porcentaje candidatas = 0.33291457286432163
########################################
Métrica: PalPer
Palabras candidatas = 248
Total analizadas actualmente = 861
Faltan etiquetar 139 palabras
Porcentaje candidatas = 0.2880371660859466
Faltan etiquetar (sin repeticiones) = 378
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 906
Total analizadas actualmente = 2160
Faltan etiquetar 2840 palabras
Porcentaje candidatas = 0.41944444444444445
########################################
Métrica: PalPer
Palabras candidatas = 919
Total analizadas actualmente = 2416
Faltan etiquetar 2

## Sin usar el log(23)

In [8]:
from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas
df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [9]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 167
Total analizadas actualmente = 761
Faltan etiquetar 239 palabras
Porcentaje candidatas = 0.21944809461235218
########################################
Métrica: Personas
Palabras candidatas = 208
Total analizadas actualmente = 724
Faltan etiquetar 276 palabras
Porcentaje candidatas = 0.287292817679558
########################################
Métrica: PalPer
Palabras candidatas = 199
Total analizadas actualmente = 778
Faltan etiquetar 222 palabras
Porcentaje candidatas = 0.25578406169665807
Faltan etiquetar (sin repeticiones) = 456
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1216
Total analizadas actualmente = 3014
Faltan etiquetar 1986 palabras
Porcentaje candidatas = 0.40345056403450563
########################################
Métrica: PalPer
Palabras candidatas = 1113
Total analizadas actualmente = 2986
Faltan etiquetar 

## Usando cálculo con normalización "vieja"


In [10]:
from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)


In [11]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [12]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 249
Total analizadas actualmente = 813
Faltan etiquetar 187 palabras
Porcentaje candidatas = 0.3062730627306273
########################################
Métrica: Personas
Palabras candidatas = 278
Total analizadas actualmente = 821
Faltan etiquetar 179 palabras
Porcentaje candidatas = 0.3386114494518879
########################################
Métrica: PalPer
Palabras candidatas = 310
Total analizadas actualmente = 939
Faltan etiquetar 61 palabras
Porcentaje candidatas = 0.3301384451544196
Faltan etiquetar (sin repeticiones) = 355
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1431
Total analizadas actualmente = 3249
Faltan etiquetar 1751 palabras
Porcentaje candidatas = 0.4404432132963989
########################################
Métrica: PalPer
Palabras candidatas = 1540
Total analizadas actualmente = 3522
Faltan etiquetar 147

## Usando el cálculo viejo CASI idéntico

No usó normalización en las dos primeras métricas

In [13]:

from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))


df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)
df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)

In [14]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.log_cantidad * df.delta_palabras
df["ival_personas"] = df.log_personas * df.delta_personas
df["ival"] = df.norm_cantidad * df.norm_personas * df.delta_palabras * df.delta_personas

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [15]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 165
Total analizadas actualmente = 757
Faltan etiquetar 243 palabras
Porcentaje candidatas = 0.21796565389696168
########################################
Métrica: Personas
Palabras candidatas = 210
Total analizadas actualmente = 722
Faltan etiquetar 278 palabras
Porcentaje candidatas = 0.29085872576177285
########################################
Métrica: PalPer
Palabras candidatas = 311
Total analizadas actualmente = 940
Faltan etiquetar 60 palabras
Porcentaje candidatas = 0.33085106382978724
Faltan etiquetar (sin repeticiones) = 457
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1219
Total analizadas actualmente = 3018
Faltan etiquetar 1982 palabras
Porcentaje candidatas = 0.4039098740888005
########################################
Métrica: PalPer
Palabras candidatas = 1545
Total analizadas actualmente = 3529
Faltan etiquetar 